In [1]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline
from jax.config import config
config.update("jax_enable_x64", True)


from functools import partial
from tqdm import tqdm
import pickle
from chainconsumer import ChainConsumer

import jax
import jax.numpy as jnp
jnp.ones([1,2])


import haiku as hk
from haiku._src.nets.resnet import ResNet18
import optax

from sbi_lens.normflow.models import AffineSigmoidCoupling, ConditionalRealNVP
from sbi_lens.gen_dataset.lensing_lognormal_dataset import LensingLogNormalDataset
import pickle
'unset XLA_FLAGS'


Populating the interactive namespace from numpy and matplotlib


/linkhome/rech/genmfd01/ulm75uc/.local/lib/python3.10/site-packages/haiku/_src/data_structures.py:37: FutureWarning: jax.tree_structure is deprecated, and will be removed in a future release. Use jax.tree_util.tree_structure instead.
  PyTreeDef = type(jax.tree_structure(None))
/gpfslocalsup/pub/anaconda-py3/2021.05/envs/python-3.10.4/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/linkhome/rech/genmfd01/ulm75uc/.local/lib/python3.10/site-packages/lenstools/utils/mpi.py:15: UserWarning: Could not import mpi4py! (if you set sys.modules['mpi4py']=None please disregard this message)
  warnings.warn(wmsg)


'unset XLA_FLAGS'

In [3]:
import logging


import tensorflow_probability as tfp; tfp = tfp.substrates.jax

# this prints a WARNING

tfp.distributions.TransformedDistribution(
    tfp.distributions.Normal(0.0, 1.0), tfp.bijectors.Identity()
)

logger = logging.getLogger("root")


class CheckTypesFilter(logging.Filter):
    def filter(self, record):
        return "check_types" not in record.getMessage()


logger.addFilter(CheckTypesFilter())

# this does not print a WARNING

tfp.distributions.TransformedDistribution(tfp.distributions.Normal(0.0, 1.0), tfp.bijectors.Identity())

<tfp.distributions.TransformedDistribution 'identityNormal' batch_shape=[] event_shape=[] dtype=float32>

# Build compressor

In [4]:
# create nf to train using mutual information
bijector_layers_compressor = [128] * 2

bijector_compressor = partial(
  AffineSigmoidCoupling, 
  layers=bijector_layers_compressor, 
  n_components=16, 
  activation=jax.nn.silu
)

NF_compressor = partial(
  ConditionalRealNVP, 
  n_layers=4, 
  bijector_fn=bijector_compressor
)

class Flow_nd_Compressor(hk.Module):
    def __call__(self, y):
        nvp = NF_compressor(2)(y)
        return nvp
nf = hk.without_apply_rng(hk.transform(lambda theta, y : Flow_nd_Compressor()(y).log_prob(theta).squeeze()))
compressor = hk.transform_with_state(lambda x : ResNet18(2)(x, is_training=True))
# ResNet 18
params_nf = nf.init(jax.random.PRNGKey(8),  0.5*jnp.ones([1,2]), 0.5*jnp.ones([1,2]))

parameters_resnet, opt_state_resnet  = compressor.init(jax.random.PRNGKey(873457568), 0.5*jnp.ones([1,128,128,5]))

parameters_compressor = hk.data_structures.merge(parameters_resnet, params_nf)



/linkhome/rech/genmfd01/ulm75uc/.local/lib/python3.10/site-packages/haiku/_src/data_structures.py:206: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, structure = jax.tree_flatten(mapping)


# Dataset

In [5]:
#### Remember to import these few lines after the previus cell to avoid CUDA error

tfd= tfp.distributions
tfb = tfp.bijectors
import tensorflow_datasets as tfds
import tensorflow as tf

In [7]:
r=tf.ones([128])

2023-04-17 12:00:17.932175: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfslocalsys/slurm/current/lib/slurm:/gpfslocalsys/slurm/current/lib
2023-04-17 12:00:17.932358: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfslocalsys/slurm/current/lib/slurm:/gpfslocalsys/slurm/current/lib
2023-04-17 12:00:17.932513: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusparse.so.11'; dlerror: libcusparse.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfslocalsys/slurm/current/lib/slurm:/gpfslocalsys/slurm/current/lib
2023-04-17 12:00:17.932555: W tensorflow/core/common_run

In [8]:
!CUDNN_PATH=$(dirname $(python -c "import nvidia.cudnn;print(nvidia.cudnn.__file__)"))
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/:$CUDNN_PATH/lib


In [9]:
r=tf.ones([128])

In [10]:
# load and create dataset
ds = tfds.load(
    'LensingLogNormalDataset/year_10_without_noise_score_density', 
    split='train', 
    data_dir = 'tensorflow_dataset'
)

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

RuntimeError: jaxlib/gpu/solver.cc:275: operation gpusolverDnDsyevjBatched_bufferSize( handle.get(), jobz, uplo, n, nullptr, n, nullptr, &lwork, params, batch) failed: cuSolver invalid value error

In [ ]:
tf.random.set_seed(1)
                           
def augmentation_with_noise(example):

      N=256
      map_size=10
      gal_per_arcmin2=27
      sigma_e=0.26
      pix_area = (map_size * 60 / N)**2  

      ex = tf.expand_dims(example['simulation'], axis = -1)
      noise = tf.random.normal(
          shape=(N, N, 1), 
          stddev=sigma_e/tf.math.sqrt(gal_per_arcmin2 * pix_area)
      )
      x = ex + noise
      x = tf.image.random_flip_left_right(x)
      x = tf.image.random_flip_up_down(x)

      return {'simulation': x, 'theta': example['theta'], 'score': example['score']}
    
def reshape(example):
      ex = example['simulation']
      x = tf.transpose(ex)
      return {'simulation': x, 'theta': example['theta'], 'score': example['score']}

In [ ]:
tfp.distributions.MultivariateNormalDiag

In [ ]:
    x = numpyro.sample(
        'y',
        dist.MultivariateNormal(loc=field,
                                covariance_matrix=jnp.diag(sigma_e**2 / (jnp.array([b.gals_per_arcmin2 for b in nz_bins]) * pix_area)))
        )
  else:

## Training

In [8]:

def loss_mse(params, mu, batch, state_resnet):
  y, opt_state_resnet = compressor.apply(
      params, state_resnet, None, batch.reshape([-1,128,128,5])
  )
  loss = jnp.mean(jnp.sum((y - mu)**2, axis=1))
  return loss, opt_state_resnet

@jax.jit
def update_compressor_with_mse(params, opt_state, mu, batch, state_resnet):
    """Single SGD update step."""
    (loss, opt_state_resnet), grads = jax.value_and_grad(loss_mse, has_aux = True)(params, mu, batch, state_resnet)
    updates, new_opt_state = optimizer_c.update(grads, opt_state)
    new_params = optax.apply_updates(params, updates)
  
    return loss, new_params, new_opt_state, opt_state_resnet

In [9]:
def loss_vmim(params, mu, batch, state_resnet):
  y, opt_state_resnet = compressor.apply(
      params, state_resnet, None, batch.reshape([-1,128,128,5])
  )
  log_prob = jax.vmap(
      lambda theta, x: nf.apply(
          params, 
          theta.reshape([1,2]), 
          x.reshape([1,2])
      ).squeeze()
  )(mu, y)
  return -jnp.mean(log_prob) , opt_state_resnet

@jax.jit
def update_compressor_with_vmim(params, opt_state, mu, batch, state_resnet):
    """Single SGD update step."""
    (loss, opt_state_resnet), grads = jax.value_and_grad(
        loss_vmim, 
        has_aux = True
    )(params, mu, batch, state_resnet)
    updates, new_opt_state = optimizer_c.update(grads, opt_state)
    new_params = optax.apply_updates(params, updates)
  
    return loss, new_params, new_opt_state, opt_state_resnet

### Define here how to train the compressor --> MSE or VMIM


In [10]:
Train= 'VMIM'

### Training compressor --> MSE or VMIM

In [11]:
with open("params_nd_compressor_{}.pkl".format(Train), 'rb') as f:
    parameters_compressor = pickle.load(f)
##################################################
with open("opt_state_resnet_{}.pkl".format(Train), 'rb') as g:
    opt_state_resnet = pickle.load(g)

In [12]:
# dataset
ds = tfds.load('LensingLogNormalDataset/year_10_without_noise_score_density', 
               split='train[:80000]', 
               data_dir = 'tensorflow_dataset')

ds = ds.repeat()
ds = ds.shuffle(1000)
ds = ds.map(reshape)
ds = ds.map(augmentation_with_noise)
ds = ds.batch(128)
ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
ds_train = iter(tfds.as_numpy(ds))

total_steps = 100_000# usually 200_000
lr_scheduler = optax.piecewise_constant_schedule(
    init_value=0.0001,
    boundaries_and_scales={int(total_steps*0.1):0.7,
                           int(total_steps*0.2):0.7,
                           int(total_steps*0.3):0.7,
                           int(total_steps*0.4):0.7,
                           int(total_steps*0.5):0.7,
                           int(total_steps*0.6):0.7,
                           int(total_steps*0.7):0.7,
                           int(total_steps*0.8):0.7,
                           int(total_steps*0.9):0.7}
)
optimizer_c = optax.adam(learning_rate=lr_scheduler)
opt_state_c = optimizer_c.init(parameters_compressor)
batch_loss=[]


In [ ]:
if Train=='MSE':
    for batch in tqdm(range(total_steps)):
        sample = next(ds_train)
        l, parameters_compressor, opt_state_c, opt_state_resnet = update_compressor_with_mse(
            parameters_compressor,
            opt_state_c, 
            sample['theta'],
            sample['simulation'], 
            opt_state_resnet
        )

        if batch % 100 == 0:
            batch_loss.append(l)
            
elif Train=='VMIM':
    for batch in tqdm(range(total_steps)):
        sample = next(ds_train)
        l, parameters_compressor, opt_state_c, opt_state_resnet = update_compressor_with_vmim(
            parameters_compressor,
            opt_state_c, 
            sample['theta'],
            sample['simulation'], 
            opt_state_resnet
        )

        if batch % 100 == 0:
            batch_loss.append(l)

In [ ]:
plt.plot(batch_loss[:500])

In [ ]:
import pickle
with open("data/params_nd_compressor_{}.pkl".format(Train), "wb") as fp:
  pickle.dump(parameters_compressor, fp)

with open("data/opt_state_resnet_{}.pkl".format(Train), "wb") as fp:
  pickle.dump(opt_state_resnet, fp)